In [ ]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D

from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

In [ ]:
def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(4, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [ ]:
# define model
model = define_model()
# create data generator
# create data generators
train_datagen = ImageDataGenerator(featurewise_center=True,rescale=1.0/255.0,width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator(featurewise_center=True,rescale=1.0/255.0)
# specify imagenet mean values for centering
train_datagen.mean = [123.68, 116.779, 103.939]
test_datagen.mean = [123.68, 116.779, 103.939]

# prepare iterator
train_it = train_datagen.flow_from_directory('/kaggle/input/data-for-rps/image_data/train/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))
test_it = test_datagen.flow_from_directory('/kaggle/input/data-for-rps/image_data/test/',
    class_mode='categorical', batch_size=64, target_size=(224, 224))
# fit model
history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
    validation_data=test_it, validation_steps=len(test_it), epochs=20, verbose=1)
# evaluate model
_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
print('> %.3f' % (acc * 100.0))
model.save('accuracy - %.3f' % (acc * 100.0) + '.h5')
# learning curves
summarize_diagnostics(history)


In [ ]:
train_it.class_indices

In [ ]:
#Loading Model
from keras.models import load_model
model = load_model("/kaggle/input/accuracy76/accuracy-76.000.h5")


In [ ]:
import cv2
import os
REV_CLASS_MAP ={0: 'none', 1: 'paper', 2 : 'rock',3:  'scissors'}
def mapper(val):
    return REV_CLASS_MAP[val]

In [ ]:
print("Enter filename\n")
filename = input("File: ")
img = cv2.imread(filename)
img = cv2.resize(img, (224, 224))
pred = model.predict(np.array([img]))
move_code = np.argmax(pred[0])
move_name = mapper(move_code)
print("Predicted: {}".format(move_name))
